# Beyond Dynamic Programming

In the first post I showed you the two main algorithms for computing optimal policies namely **value iteration** and **policy iteration**. We modelled the environment as a Markov decision process (MDP), and we used a transition model to describe the probability of moving from one state to the other. The transition model was stored in a matrix `T` and used to find the utility function $U^{\ast}$ and the best policy $\pi^{\ast}$. Here we must be **careful with the mathematical notation**. In the book of Sutton and Barto, the utility function is called value function or state-value function and is indicated with the letter $V$. In order to keep everything uniform I will use the notation of Russels and Norvig which uses the letter $U$ to identify the utility function. The two notations have the same meaning and they define the value of a state as the expected cumulative future discounted reward starting from that state. The reader should get used to different notations, it is a good form of mental gymnastics.

Now I would like to give a proper definition of **model-free reinforcement learning** and in particular of **passive** and **active** reinforcement learning. In model-free reinforcement learning, the first thing we miss is a **transition model**. In fact the name model-free stands for transition-model-free. The second thing we miss is the **reward function** $R(s)$ which gives to the agent the reward associated to a particular state. In the passive approach we have a **policy** $\pi$ which the agent can use to move in the environment. In state $s$ the agent always produce the action $a$ given by the policy $\pi$. The goal of the agent in **passive** reinforcement learning is to learn the utility function $U^\pi (s)$. Sutton and Barto called this case MC for **prediction**. It is also possible to estimate the optimal policy while moving in the environment. In this case we are in an **active** case and using the words of Sutton and Barto we would say that we are applying MC for **control** estimation. Here I will use again the example of the **cleaning robot** from the first post but with a different setup.

<img src="files/figures/reinforcement_learning_model_free_passive_simple_world.png" style="width: 500px;" />

The robot is in a $4 \times 3$ world with an unknown transition model. The only information about the environment is the **state availability**. Since the robot does not have the reward function it does not know which state contains the charging station (+1) and which state contains the stairs (-1). Only in the passive case the robot has a **policy** that can follow to move in the world. Finally the **transition model**, since the robot does not know what it is going to happen after each action it can only give unknown probabilities to each possible outcome. To summarise, in the **passive** case this is what we have:

1. Set of possible States: $S=\{ s_0, s_1, \ldots, s_m \}$
2. Initial State: $s_0$
3. Set of possible Actions: $A=\{ a_0, a_1, \ldots, a_n \}$
4. The policy $\pi$

In passive reinforcement learning our **objective** is to use the available information to **estimate the utility function**. How to do it?

The first thing the robot can do is to **estimate the transition model**, moving in the environment and keeping track of the number of times an action has been correctly executed. Once the transition model is available the robot can use either value iteration or policy iteration to get the utility function. In this sense, there are different techniques which can find out the transition model making use of Bayes rule and maximum likelihood estimation. Russell and Norvig mention these techniques in chapter 21.2.2 (Bayesian reinforcement learning). The problem of this approach should be evident: **estimating the values of a transition model can be expensive**. In our $3 \times 4$ world it means to estimate the values for a $12 \times 12 \times 4$ (states x states x actions) table. Moreover certain actions and some states can be extremely unlikely, making the entries in the transition table hard to estimate. Here I will focus on another technique which directly estimates the utility function without using the transition model, I am talking about the **Monte Carlo method**.

# The Monte Carlo Method

The Monte Carlo (MC) method was used for the first time in 1930 by **Enrico Fermi** who was studying neutron diffusion. Fermi did not publish anything on it, the modern version is due to **Stanislaw Ulam** who invented it during the 1940s at Los Alamos. The idea behind MC is simple: using randomness to solve problems. For example it is possible to use MC to estimate a multidimensional definite integral, a technique which is called **MC integration**. In artificial intelligence we can use MC tree search to find the best move in a game. The **DeepMind AlphaGo** defeated the Go world champion using MC tree search combined with convolutional neural networks and deep reinforcement learning. Later on in this series we will discover how it was possible. The **advantages of MC methods over the dynamic programming approach** are the following:

1. MC allow learning optimal behaviour **directly from interaction** with the environment.
2. It is easy and efficient to **focus** MC methods on small **subset of the states**.
3. MC can be used with **simulations** (sample models).

During the post I will analyse the first two points. The **third point** is less intuitive. In many applications it is easy to simulate episodes but it can be extremely difficult to construct the transition model required by the dynamic programming techniques. In all these cases the MC method rules. Now let's go back to our **cleaning robot** and let's see what does it mean to apply the MC method to this scenario. As each step it records the reward obtained and saves a history of all the states visited until reaching a terminal state. We call an **episode** the sequence of states from the starting state to the terminal state. Now let's suppose that our robot recorded the foolwoing three episodes:

<img src="files/figures/reinforcement_learning_model_free_monte_carlo_three_episodes_fast.gif" style="width: 500px;" />

The robot followed its internal policy but **an unknown transition model perturbed the trajectory** leading to undesired states. In the first and second episode, after some fluctuation the robot eventually reached the terminal state obtaining a positive reward. In the third episode, the robot moved along a wrong path reaching the stairs and falling down (reward: -1.0). The following is another representation of the three episodes:

<img src="files/figures/reinforcement_learning_model_free_monte_carlo_three_episodes_linear.png" style="width: 500px;" />

Each occurrence of a state during the episode is called **visit**. The concept of visit is important because it permits defining two different MC approaches:

1. **First-Visit MC**: $U^\pi (s)$ is defined as the average of the returns following the *first visit* to $s$ in a set of episodes.
2. **Every-Visit MC**: $&^\pi (s)$ is defined as the average of the returns following *all the visit* to $s$ in a set of episodes.

I will **focus only on the First-Visit MC method in this article**. What does **return** means? The return is the sum of discounted reward. I already presented the return in the first article when I introduced the Bellman equation and the utility of a state history.

$$Return(s) = \sum_{t=0}^{\infty}\gamma^t R(S_t)$$

There is nothing new. We have the discount factor $\gamma$, the reward function $R(s)$ and $S_t$ the state reached at time $t$. We can calculate the return for the state (1, 1) of the first episode, with $\gamma=0.9$, as follow:

<img src="files/figures/reinforcement_learning_model_free_return_first_episode.png" style="width: 500px;" />

The return for the first episode is 0.27. Following the same procedure we get the same result for the second episode. For the third episode we get a different return -0.79. After the three episodes we came out with three different returns: 0.27, 0.27, -0.79. **How to use the returns to estimate the utilities**? I will now introduce the core equation used in the MC method, which give the utility of a state following the policy $\pi$:

$$U^\pi (s) = \mathbb{E}\left[ \sum_{t=0}^{\infty} \gamma^t R(S_t) \right]$$

If you compare this equation with the equation used to calculate the return you will see only one difference: to obtain the utility function we take the **expectation of the returns**. That's it. To find the utility of a state we need to calculate the expectation of the returns for that state. In our example after only three episodes the approximated utility for the state (1, 1) is: $\frac{0.27+0.27-0.79}{3} = -0.08$. However, an estimation based only on three episodes is inaccurate. We need more episodes in order to get the true value. Why do we need more episodes?

**Here is where the MC terminology steps into**. We can define $S_t$ to be a **discrete random variable** which can assume all the available states with a certain probability. Every time our robot steps into a state is like if we are picking a value for the random variable $S_t$. For each state of each episode we can calculate the return and store it in a list. Repeating this process for a large number of times is **guaranteed to converge to the true utility**. How is that possible? This is the result of a famous theorem known as the **law of large number**. Understanding the law of large number is crucial. Rolling a six-sided dice produces one of the numbers 1, 2, 3, 4, 5, or 6, each with equal probability. The **expectation** is 3.5 and can be calculated as the arithmetic mean: $\frac{(1+2+3+4+5+6)}{6}=3.5$. Using a MC approach we can obtain the same value, let's do it in Python:

```python
import numpy as np
# Throwing a dice for N times and evaluating the expectation
dice = np.random.randint(low=1, high=7, size=3)
print("Expectation (rolling 3 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=10)
print("Expectation (rolling 10 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=100)
print("Expectation (rolling 100 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=1000)
print("Expectation (rolling 1000 times): " + str(np.mean(dice)))
dice = np.random.randint(low=1, high=7, size=100000)
print("Expectation (rolling 100000 times): " + str(np.mean(dice)))
```

```
Expectation (rolling 3 times): 4.0
Expectation (rolling 10 times): 2.9
Expectation (rolling 100 times): 3.47
Expectation (rolling 1000 times): 3.481
Expectation (rolling 100000 times): 3.49948
```

As you can see the estimation of the expectation converges to the true value of 3.5. What we are doing in MC reinforcement learning is exactly the same but in this case we want to **estimate the utility for each state based on the return of each episode**. As for the dice, more episodes we take into account, more accurate our estimation will be.

# Python Implementation